# Lets Start NLP Module

In [2]:
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from transformers import DistilBertModel, DistilBertTokenizer
from sklearn import metrics
from tqdm import tqdm
import numpy as np
import re

# print(data.columns)
# Extract 'problems' and 'findings' columns
df= pd.read_csv("indiana_reports.csv")[["Problems","findings"]]
#print df train element by element
print(df.iloc[0])




Problems                                               normal
findings    The cardiac silhouette and mediastinum size ar...
Name: 0, dtype: object


In [2]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

print(device)
#Also print gpu name
if device == 'cuda':
    print(cuda.get_device_name(0))

cuda
NVIDIA GeForce RTX 3060 Laptop GPU


In [3]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

def remove_numbers(text):
    text = ''.join([i for i in text if not i.isdigit()])         
    return text

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_username(text):
    url = re.compile(r'@[A-Za-z0-9_]+')
    return url.sub(r'',text)

def pre_process_text(text):
    text = remove_URL(text)
    text = remove_numbers(text)
    text = remove_html(text)
    text = remove_username(text)
    return " ".join(text.split())

In [4]:
df.dropna(subset=['findings'], inplace=True)  # Remove rows with NaN in 'findings'
df = df[df['findings'].str.strip().astype(bool)]  # Remove rows with empty or whitespace 'findings'

In [5]:
print(df['Problems'].value_counts())
print(df['findings'].value_counts())

Problems
normal                                                      1197
No Indexing                                                   87
Lung                                                          79
Calcified Granuloma                                           72
Thoracic Vertebrae                                            59
                                                            ... 
Pulmonary Atelectasis;Foreign Bodies;Density                   1
Calcified Granuloma;Lung;Markings                              1
Lung;Opacity;Markings                                          1
Hernia, Diaphragmatic;Bone Diseases, Metabolic;Deformity       1
Opacity;Granuloma                                              1
Name: count, Length: 1244, dtype: int64
findings
The heart and lungs have XXXX XXXX in the interval. Both lungs are clear and expanded. Heart and mediastinum normal.                                                                                                               

# Limiting 100 characters for pipeline checks

In [6]:
# df['findings'] = df['findings'].str[:200]

In [7]:
df['Problems'] = df['Problems'].str.replace(';', ' and ')

In [8]:
from transformers import T5Tokenizer, AutoTokenizer
from sklearn.model_selection import train_test_split

# Splitting source and target into train and eval sets (80-20 split)

# Assuming df is your DataFrame
source = df['findings'].tolist()
target = df['findings'].tolist()

source_train, source_eval, target_train, target_eval = train_test_split(
    source, target, test_size=0.2, random_state=42
)

# Convert the data into strings
# source_train_str = '\n'.join(source_train)
# target_train_str = '\n'.join(target_train)
# source_eval_str = '\n'.join(source_eval)
# target_eval_str = '\n'.join(target_eval)

max_length = 128  # or any other desired maximum length
# Initialize the T5 tokenizer
# Initialize the BioClinicalBERT tokenizer
from transformers import AutoTokenizer, BioGptForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")



# Tokenize the source and target texts with max_length
tokenized_source = tokenizer(source_train, padding=True, truncation=True, return_tensors="pt", max_length=max_length)
tokenized_target = tokenizer(target_train, padding=True, truncation=True, return_tensors="pt", max_length=max_length)

eval_source = tokenizer(source_eval, padding=True, truncation=True, return_tensors="pt", max_length=max_length)
eval_target = tokenizer(target_eval, padding=True, truncation=True, return_tensors="pt", max_length=max_length)

print('Tokenized source has shape', tokenized_source['input_ids'].shape)
print('Tokenized target has shape', tokenized_target['input_ids'].shape)

print('Eval source has shape', eval_source['input_ids'].shape)
print('Eval target has shape', eval_target['input_ids'].shape)


Tokenized source has shape torch.Size([2669, 128])
Tokenized target has shape torch.Size([2669, 128])
Eval source has shape torch.Size([668, 128])
Eval target has shape torch.Size([668, 128])


In [9]:
print(tokenized_source)
print(tokenized_target)

{'input_ids': tensor([[    2,    18,  3800,  ...,     1,     1,     1],
        [    2,  5389,     8,  ...,     1,     1,     1],
        [    2,    18,  6439,  ...,     1,     1,     1],
        ...,
        [    2, 12518,  1344,  ...,     1,     1,     1],
        [    2, 15137,  1419,  ...,     1,     1,     1],
        [    2,   271,    31,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
{'input_ids': tensor([[    2,    18,  3800,  ...,     1,     1,     1],
        [    2,  5389,     8,  ...,     1,     1,     1],
        [    2,    18,  6439,  ...,     1,     1,     1],
        ...,
        [    2, 12518,  1344,  ...,     1,     1,     1],
        [    2, 15137,  1419,  ...,     1,     1,     1],
        [    2,   271,    31,  ...,     1,     1,     1]]), 'attentio

In [10]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, source_input_ids, source_attention_mask, target_input_ids, target_attention_mask):
        self.source_input_ids = source_input_ids
        self.source_attention_mask = source_attention_mask
        self.target_input_ids = target_input_ids
        self.target_attention_mask = target_attention_mask

    def __len__(self):
        return len(self.source_input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.source_input_ids[idx],
            'attention_mask': self.source_attention_mask[idx],
            'labels': self.target_input_ids[idx],
            'labels_attention_mask': self.target_attention_mask[idx]
        }

# Assuming you have the tokenized tensors: tokenized_source and tokenized_target
train_dataset = CustomDataset(
    tokenized_source['input_ids'],
    tokenized_source['attention_mask'],
    tokenized_target['input_ids'],
    tokenized_target['attention_mask']
)
eval_dataset = CustomDataset(
    eval_source['input_ids'],
    eval_source['attention_mask'],
    eval_target['input_ids'],
    eval_target['attention_mask']
)


In [11]:
print(train_dataset)
print(eval_dataset)




# Checks

In [12]:
# Print out tokens to see if they look correct
sample_idx = 12  # Example index, you can change it to view different samples

print("Sample Source Text Tokenized:")
print(tokenizer.convert_ids_to_tokens(tokenized_source['input_ids'][sample_idx]))

print("\nSample Target Text Tokenized:")
print(tokenizer.convert_ids_to_tokens(tokenized_target['input_ids'][sample_idx]))

Sample Source Text Tokenized:
['</s>', 'The</w>', 'lungs</w>', 'are</w>', 'hyperex', 'p', 'anded</w>', ',</w>', 'consistent</w>', 'with</w>', 'COPD</w>', '.</w>', 'Mild</w>', 'cardi', 'omegaly</w>', '.</w>', 'No</w>', 'focal</w>', 'lung</w>', 'consolidation</w>', '.</w>', 'No</w>', 'pneumothorax</w>', 'or</w>', 'pleural</w>', 'effusion</w>', '.</w>', 'Pulmonary</w>', 'vascularity</w>', 'is</w>', 'within</w>', 'normal</w>', 'limits</w>', '.</w>', 'Mild</w>', 'degenerative</w>', 'changes</w>', 'of</w>', 'the</w>', 'thoracic</w>', 'spine</w>', '.</w>', 'Aortic</w>', 'calcifications</w>', 'consistent</w>', 'with</w>', 'atherosclerotic</w>', 'disease</w>', '.</w>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'

# Finetuning

In [13]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [14]:
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoModelForTokenClassification
from torch.utils.data import Dataset

# Initialize the BioClinicalBERT model
model = BioGptForCausalLM.from_pretrained("microsoft/biogpt")


In [15]:

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy="epoch",
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Use the created dataset here
    eval_dataset=eval_dataset
    # eval_dataset=train_dataset
)

# Train the model
trainer.train()

# Save the trained model to a directory
model.save_pretrained("/FYP_DATASET/results/")


  0%|          | 0/5010 [00:00<?, ?it/s]

{'loss': 6.8853, 'learning_rate': 1e-05, 'epoch': 0.3}
{'loss': 1.1273, 'learning_rate': 2e-05, 'epoch': 0.6}
{'loss': 0.5702, 'learning_rate': 3e-05, 'epoch': 0.9}


  0%|          | 0/84 [00:00<?, ?it/s]

{'eval_loss': 0.4948144853115082, 'eval_runtime': 18.305, 'eval_samples_per_second': 36.493, 'eval_steps_per_second': 4.589, 'epoch': 1.0}
{'loss': 0.4803, 'learning_rate': 4e-05, 'epoch': 1.2}
{'loss': 0.4647, 'learning_rate': 5e-05, 'epoch': 1.5}
{'loss': 0.4293, 'learning_rate': 4.8891352549889134e-05, 'epoch': 1.8}


  0%|          | 0/84 [00:00<?, ?it/s]

...

{'eval_loss': 0.5696318745613098, 'eval_runtime': 15.5262, 'eval_samples_per_second': 43.024, 'eval_steps_per_second': 5.41, 'epoch': 13.0}
{'loss': 0.0723, 'learning_rate': 6.7627494456762755e-06, 'epoch': 13.17}
{'loss': 0.0714, 'learning_rate': 5.65410199556541e-06, 'epoch': 13.47}
{'loss': 0.0721, 'learning_rate': 4.5454545454545455e-06, 'epoch': 13.77}


  0%|          | 0/84 [00:00<?, ?it/s]

{'eval_loss': 0.5746690630912781, 'eval_runtime': 15.4816, 'eval_samples_per_second': 43.148, 'eval_steps_per_second': 5.426, 'epoch': 14.0}
{'loss': 0.0717, 'learning_rate': 3.436807095343681e-06, 'epoch': 14.07}
{'loss': 0.0681, 'learning_rate': 2.328159645232816e-06, 'epoch': 14.37}
{'loss': 0.0692, 'learning_rate': 1.2195121951219514e-06, 'epoch': 14.67}
{'loss': 0.0694, 'learning_rate': 1.1086474501108648e-07, 'epoch': 14.97}


  0%|          | 0/84 [00:00<?, ?it/s]

{'eval_loss': 0.5771459937095642, 'eval_runtime': 15.45, 'eval_samples_per_second': 43.236, 'eval_steps_per_second': 5.437, 'epoch': 15.0}
{'train_runtime': 11134.2677, 'train_samples_per_second': 3.596, 'train_steps_per_second': 0.45, 'train_loss': 0.32219941303163707, 'epoch': 15.0}


In [16]:
from transformers import AutoModelForSequenceClassification

loaded_model = AutoModelForSequenceClassification.from_pretrained("/FYP_DATASET/results/")
print(loaded_model)


Some weights of BioGptForSequenceClassification were not initialized from the model checkpoint at /FYP_DATASET/results/ and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BioGptForSequenceClassification(
  (biogpt): BioGptModel(
    (embed_tokens): Embedding(42384, 1024, padding_idx=1)
    (embed_positions): BioGptLearnedPositionalEmbedding(1026, 1024)
    (layers): ModuleList(
      (0-23): 24 x BioGptDecoderLayer(
        (self_attn): BioGptAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (activation_fn): GELUActivation()
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layer_norm): LayerNorm((1024,)

# Random Input to Check Model

In [17]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# df= pd.read_csv("indiana_reports.csv")[["findings"]]
# df.dropna(subset=['findings'], inplace=True)  # Remove rows with NaN in 'findings'
# df = df[df['findings'].str.strip().astype(bool)]  # Remove rows with empty or whitespace 'findings'

# all_sentences = []
# for finding in df['findings']:
#     sentences = finding.split('\n')  # Split into sentences assuming '.' is the sentence delimiter
#     # Filter sentences by length (25 characters) and remove any leading/trailing spaces
#     filtered_sentences = [sentence.strip() for sentence in sentences if len(sentence.strip()) == 25]
#     all_sentences.extend(filtered_sentences)

# # Ensure there are enough sentences of the exact length
# if len(all_sentences) >= 5:
#     # Randomly select 5 sentences
#     random_sentences = np.random.choice(all_sentences, size=5, replace=False)
# else:
#     print("Not enough sentences of exactly 25 characters. Adjusting selection criteria.")
#     # Handle the case where there are not enough sentences of the exact length, e.g., by choosing shorter ones or repeating some
#     # This is a placeholder for whatever fallback logic you deem appropriate
#     random_sentences = np.random.choice(all_sentences, size=5, replace=True)  # Example fallback


# # Load the language model and tokenizer
# model_directory = "/FYP_DATASET/results/"
# loaded_model = AutoModelForCausalLM.from_pretrained(model_directory)
# tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")

# # Get user input
# user_input = random_sentences
# # user_input=user_input.replace(';', ' and ')

# # Tokenize the user input
# input_ids = tokenizer.encode(user_input, return_tensors="pt")

# # Generate output from the language model
# outputs = []

# for user_input in random_sentences:
#     # Tokenize the user input
#     input_ids = tokenizer.encode(user_input, return_tensors="pt")

#     # Generate output from the language model
#     output = loaded_model.generate(
#         input_ids,
#         max_length=1000,  # Adjust the maximum length here
#         num_return_sequences=1,  # Increase if you need multiple sequences
#         pad_token_id=tokenizer.eos_token_id,  # Padding token ID
#         do_sample=True,  # Enable sampling
#         temperature=0.7,  # Control randomness of sampling
#         top_k=50,  # Filter top-k tokens to sample from
#         top_p=0.95,  # Filter cumulative probability for top-p sampling
#         repetition_penalty=1.0,  # Adjusts for repeating tokens
#     )

#     # Add the generated output to the list
#     outputs.append((user_input, output[0]))

# # Now, decode and print all outputs
# for user_input, output in outputs:
#     decoded_output = tokenizer.decode(output, skip_special_tokens=True)
#     print("User input:", user_input)
#     print("Generated output:", decoded_output)
#     print("---------------------------------------------------")


# Custom Input to Check Model

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the language model and tokenizer
model_directory = "/FYP_DATASET/results/"

loaded_model = AutoModelForCausalLM.from_pretrained(model_directory)
tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead. Inference might be slow.")

# Move the model to the GPU
loaded_model = loaded_model.to(device)

# Get user input
user_input = input("Enter your input text: ")
user_input=user_input.replace(';', ' and ')

# Tokenize the user input
input_ids = tokenizer.encode(user_input, return_tensors="pt")

# Move the input_ids to the same device as the model
input_ids = input_ids.to(device)


# Generate output from the language model
output = loaded_model.generate(
    input_ids,
    max_length=200,  # Adjust the maximum length here
    num_return_sequences=1,  # Increase if you need multiple sequences
    pad_token_id=tokenizer.eos_token_id,  # Padding token ID
    do_sample=True,  # Enable sampling
    temperature=0.7,  # Control randomness of sampling
    top_k=50,  # Filter top-k tokens to sample from
    top_p=0.95,  # Filter cumulative probability for top-p sampling
    repetition_penalty=1.0,  # Adjusts for repeating tokens
)

# Decode the output tokens to text
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print("User input:", user_input)
# Display the generated text
print("Generated output:", decoded_output)


Using GPU: NVIDIA GeForce RTX 3060 Laptop GPU
User input: Acute Cardiomegaly
Generated output: Acute Cardiomegaly. Mediastinal contours are normal limits. Increased interstitial opacities. No pneumothorax or large pleural effusion. No acute osseous abnormality. cy. TIPA.
